In [ ]:
! pip install roboflow

In [26]:
from roboflow import Roboflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2

In [27]:
from roboflow import Roboflow
rf = Roboflow(api_key="hiXB6MTQOH0hlxROddmK")
project = rf.workspace("jeevan-sendur-g-workspace").project("board-kzxer")
version = project.version(3)
dataset = version.download("yolov8")



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to board-3 in yolov8:: 100%|██████████| 1650/1650 [00:00<00:00, 3138.42it/s]


In [28]:
! pip install ultralytics

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [33]:
from ultralytics import YOLO

model = YOLO("/content/runs/pose/train2/weights/best.pt")

In [32]:
results = model.train(data="/content/board-3/data.yaml", epochs=100, imgsz=640, device=0)

Ultralytics 8.3.55 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=pose, mode=train, model=/content/runs/pose/train2/weights/best.pt, data=/content/board-3/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True

train: Scanning /content/board-3/train/labels.cache... 780 images, 0 backgrounds, 0 corrupt: 100%|██████████| 780/780 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/board-3/valid/labels.cache... 30 images, 0 backgrounds, 0 corrupt: 100%|██████████| 30/30 [00:00<?, ?it/s]


Plotting labels to runs/pose/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 87 weight(decay=0.0), 97 weight(decay=0.0005), 96 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/pose/train3
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


...

     28/100       2.4G     0.3946     0.5993     0.1644     0.2615      0.905         19        640: 100%|██████████| 49/49 [00:14<00:00,  3.27it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.24it/s]

                   all         30         30      0.996          1      0.995      0.932      0.996          1      0.995      0.976



      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


     29/100       2.4G     0.3902     0.4628     0.1463     0.2405     0.9045         35        640:   4%|▍         | 2/49 [00:01<00:28,  1.63it/s]


KeyboardInterrupt: 

In [21]:
model = YOLO("/content/runs/pose/train/weights/best.pt")

results = model("/content/testingpic.jpg")


image 1/1 /content/testingpic.jpg: 640x480 1 board, 12.1ms
Speed: 3.1ms preprocess, 12.1ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 480)


In [36]:
results = model("/content/testingpic.jpg")


image 1/1 /content/testingpic.jpg: 1024x768 1 board, 56.3ms
Speed: 6.2ms preprocess, 56.3ms inference, 1.9ms postprocess per image at shape (1, 3, 1024, 768)


In [22]:
for result in results:
    keypoints = result.keypoints
    result.save(filename="result.jpg")

In [37]:
import cv2
import numpy as np

def visualize_keypoints(results, thickness=7, radius=5, line_thickness=6, point_color=(0, 255, 0), line_color=(255, 255, 0)):
    for result in results:
        # Get the original image
        img = result.orig_img.copy()

        # Get keypoints
        keypoints = result.keypoints.data

        if len(keypoints):
            for kpts in keypoints:
                # Store valid keypoints
                valid_points = []

                # Draw keypoints and collect valid ones
                for x, y, conf in kpts:
                    if conf > 0:
                        x_coord, y_coord = int(x), int(y)
                        valid_points.append((x_coord, y_coord))
                        cv2.circle(img, (x_coord, y_coord), radius, point_color, thickness)

                # Connect points in a square if we have 4 points
                if len(valid_points) == 4:
                    # Connect points in square formation
                    for i in range(4):
                        # Connect to next point (last point connects to first)
                        next_idx = (i + 1) % 4
                        cv2.line(img, valid_points[i], valid_points[next_idx],
                                line_color, line_thickness)

        # Save the result
        cv2.imwrite('keypoints_square.jpg', img)

# Usage:
for result in results:
    visualize_keypoints(
        results=[result],
        thickness=7,      # Keypoint circle thickness
        radius=5,         # Keypoint circle radius
        line_thickness=6, # Thickness of connecting lines
        point_color=(0, 255, 0),   # Green dots
        line_color=(255, 255, 0)   # Yellow lines
    )

In [ ]:
model.train(
        # Data Configuration
        data='/content/board-3/data.yaml',          # Path to your data.yaml
        imgsz=1024,               # Larger image size for better accuracy

        # Training Duration
        epochs=100,               # Extended training time
        patience=50,             # More patience for early stopping

        # Learning Rate Configuration
        lr0=0.001,               # Initial learning rate
        lrf=0.0001,              # Final learning rate

        # Loss Weights          # Increased keypoint loss weight

        # Data Augmentation (reduced for better pose accuracy)
        scale=0.2,               # Reduced scale augmentation
        degrees=5.0,             # Minimal rotation
        translate=0.1,           # Minimal translation
        mosaic=0.2,              # Reduced mosaic augmentation
        mixup=0.0,               # Disabled mixup

        # Other Optimizations
        batch=8,                 # Adjusted batch size for larger images
        device=0,                # Use GPU

        # Save Settings
        save_period=10,          # Save every 10 epochs
        exist_ok=True,           # Overwrite existing experiment

        # Additional parameters for stability
        warmup_epochs=5,         # Longer warmup
        cos_lr=True,             # Cosine LR scheduler
        overlap_mask=True,       # Better mask overlapping
        )


In [39]:
import torch
torch.save(model.state_dict(),'model.pt')